# Spela v75

In [26]:
!pip install selenium

import pandas as  pd 
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import scraping as scr

import sys
sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\version 3')
import V75_scraping as vs

import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)

## Do the scraping - loop över avdelningar

In [27]:
def do_scraping(driver_s, omg_links): #get data from web site
    print(omg_links)
    logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)
    
    vdict = {'datum':[], 'bana':[], 'avd':[], 'startnr':[],'häst':[],'ålder':[],'kön':[],'kusk':[],'lopp_dist':[],
    'start':[],'dist':[],'pris':[],'spår':[],'streck':[],'vodds':[],'podds':[],'kr':[], 
    'h1_dat':[],'h1_bana':[],'h1_kusk':[],'h1_plac':[],'h1_dist':[],'h1_spår':[],'h1_odds':[],'h1_pris':[],'h1_kmtid':[],
    'h2_dat':[],'h2_bana':[],'h2_kusk':[],'h2_plac':[],'h2_dist':[],'h2_spår':[],'h2_odds':[],'h2_pris':[],'h2_kmtid':[],
    'h3_dat':[],'h3_bana':[],'h3_kusk':[],'h3_plac':[],'h3_dist':[],'h3_spår':[],'h3_odds':[],'h3_pris':[],'h3_kmtid':[],
    'h4_dat':[],'h4_bana':[],'h4_kusk':[],'h4_plac':[],'h4_dist':[],'h4_spår':[],'h4_odds':[],'h4_pris':[],'h4_kmtid':[],
    'h5_dat':[],'h5_bana':[],'h5_kusk':[],'h5_plac':[],'h5_dist':[],'h5_spår':[],'h5_odds':[],'h5_pris':[],'h5_kmtid':[],
    }

    start_time = time.perf_counter()
    # Hela omgången (driver_s)
    dat = omg_df.Link[0].split('spel/')[1][0:10]
    game_tab=driver_s.find_elements_by_class_name('game-table')[1:]                       ## alla lopp utan rubriker
    comb=driver_s.find_elements_by_class_name('race-combined-info')                 ## alla bana,dist,start
    priser=driver_s.find_elements_by_class_name('css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')
    priser = [p.text for p in priser if 'Pris:' in p.text]                          ## alla lopps priser

    # ett lopp (de häst-relaterade som inte kan bli 'missing' tar jag direkt på loppnivå)
    for anr,avd in enumerate(game_tab):
        dat = omg_links[0].split('spel/')[1][0:10]  #gräv ut datum  
        logging.warning(dat+' avd: '+str(avd))

        bana=comb[anr].text.split('\n')[0]
        lopp_dist= comb[anr].text.split('\n')[1].split(' ')[0][:-1]
        start = comb[anr].text.split('\n')[1].split(' ')[1]

        pris=priser[0].split('-')[0].split(' ')[1]
        
        names     = avd.find_elements_by_class_name("horse-name")                         ## alla hästar/kön/åldet i loppet
        voddss    = avd.find_elements_by_class_name("vOdds-col")[1:]                      ## vodds i loppet utan rubrik
        poddss    = avd.find_elements_by_class_name("pOdds-col")[1:]                      ## podds i loppet utan rubrik
        rader     = avd.find_elements_by_class_name("startlist__row")                     ## alla rader i loppet
        strecks   = avd.find_elements_by_class_name("betDistribution-col")[1:]            ## streck i loppet  utan rubrik

        print('AVD',anr+1,bana,lopp_dist,start,end=' ')
        
        ## history ##
        hist=avd.find_elements_by_class_name("start-info-panel")  # all history för loppet

        for r,rad in enumerate(rader):
            # en häst
            logging.warning(r)
    
            vdict['datum'].append(dat)
            vdict['bana'].append(bana)
            vdict['avd'].append(anr+1)
            vdict['startnr'].append(r+1)
            vdict['häst'].append(names[r].text)
            vdict['start'].append(start)
            vdict['lopp_dist'].append(lopp_dist)
            vdict['pris'].append(pris)
            vdict['vodds'].append(voddss[r].text)            ##  vodds i lopp 1 utan rubrik
            vdict['podds'].append(poddss[r].text)            ##  podds i lopp 1 utan rubrik
            vdict['streck'].append(strecks[r].text)          ##  streck i lopp 1  utan rubrik
            ### kusk måste tas på hästnivå (pga att driver-col finns också i hist)
            vdict['kusk'].append(rad.find_elements_by_class_name("driver-col")[0].text)                         
            vdict['ålder'].append(rad.find_elements_by_class_name("horse-age")[0].text)                         
            vdict['kön'].append(rad.find_elements_by_class_name("horse-sex")[0].text)                          
            vdict['kr'].append(rad.find_elements_by_class_name("earningsPerStart-col")[0].text)  ##  kr/start i lopp 1 utan rubrik
            dist_sp = rad.find_elements_by_class_name("postPositionAndDistance-col")    ##  dist och spår i lopp 1 utan rubrik
            vdict['dist'].append(dist_sp[0].text.split(':')[0])
            vdict['spår'].append(dist_sp[0].text.split(':')[1])
            
            ### history
            h_dates =hist[r].find_elements_by_class_name('date-col')[1:]
            h_kuskar=hist[r].find_elements_by_class_name('driver-col')[1:]
            h_banor =hist[r].find_elements_by_class_name('track-col')[1:]
            h_plac  = hist[r].find_elements_by_class_name('place-col')[1:]  ## obs varannan rad (10 rader)
            
            h_dist = hist[r].find_elements_by_class_name('distance-col')[1:]
            h_spår = hist[r].find_elements_by_class_name('position-col')[1:]
            h_kmtid= hist[r].find_elements_by_class_name('kmTime-col')[1:]
            h_odds = hist[r].find_elements_by_class_name('odds-col')[1:]
            h_pris = hist[r].find_elements_by_class_name('firstPrize-col')[1:]
            
            for h,d in enumerate(h_dates)  : 
                fld='h'+str(h+1)+'_'
                vdict[fld+'dat'].append(d.text)
                vdict[fld+'bana'].append(h_banor[h].text)
                vdict[fld+'kusk'].append(h_kuskar[h].text)
                vdict[fld+'plac'].append(h_plac[h].text)
                vdict[fld+'dist'].append(h_dist[h].text)
                vdict[fld+'spår'].append(h_spår[h].text)
                vdict[fld+'kmtid'].append(h_kmtid[h].text)
                vdict[fld+'odds'].append(h_odds[h].text)
                vdict[fld+'pris'].append(h_pris[h].text)
            
            print('.',end='')
        print()
        
    print('\ndet tog',round(time.perf_counter() - start_time,3),'sekunder')
        
    return vdict
        

## Start the scraping

In [28]:
df=pd.DataFrame()
def scraping(omg_df, driver_s): #get data from web site
    global df 
    global komplett
    #Läs in alla lopp från startlista
    komplett = do_scraping(driver_s, omg_df.Link)
    df = pd.DataFrame(komplett)
    print(df.shape)
    # df.columns=['datum','bana','avd','startnr','häst','kön','ålder', 'kusk', 'spår', 'streck','vodds', 'podds', 'pris', 'start','dist','lopp_dist','kr',  
    #     'h1_dat', 'h1_bana', 'h1_kusk', 'h1_plac', 'h1_dist','h1_spår','h1_kmtid','h1_odds','h1_pris',
    #     'h2_dat', 'h2_bana', 'h2_kusk', 'h2_plac', 'h2_dist','h2_spår','h2_kmtid','h2_odds','h2_pris',
    #     'h3_dat', 'h3_bana', 'h3_kusk', 'h3_plac', 'h3_dist','h3_spår','h3_kmtid','h3_odds','h3_pris',
    #     'h4_dat', 'h4_bana', 'h4_kusk', 'h4_plac', 'h4_dist','h4_spår','h4_kmtid','h4_odds','h4_pris',
    #     'h5_dat', 'h5_bana', 'h5_kusk', 'h5_plac', 'h5_dist','h5_spår','h5_kmtid','h5_odds','h5_pris'
    #     ]

    # fixa kolumner och NaN's 
    strukna = df[df.vodds=='EJ'][['datum','avd','startnr','häst','vodds']]
    df = df[df.vodds != 'EJ']  # Ta bort alla strukna
    df['pris']=df.pris.astype(float)
    import fixa_mer_features as ff2
    print(f"\n\ndf.shape före fixa_mer_features: {df.shape}\n")
    df = ff2.fixa_mer_features(df, True)
    print(f"\n\ndf.shape efter fixa_mer_features: {df.shape}")

    df.ålder = df.ålder.astype('int')
    
    # strecken
    print(f'streck före med NaNs: {df.streck.isna().sum()}')
    #df.streck = df.streck.str.replace('%','')
    #df.streck = df.streck.replace(' ',np.nan)
    df.streck = df.streck.astype('float')
    print(f'streck efter med NaNs: {df.streck.isna().sum()}')


    return df, strukna


## Spara undan vodds och proba
## 30 min innan, 15 min innan, 5 min innan, efter att hela omgången är färdig
5 min innan är det faktiska spelet. Efter omgången har vi odds som inte var kända när kupongen lämnas in

In [29]:
def addera_vodds(df,vecko_df,dat,fas):
    # Uppdatera vecko_df 4 värden på vodds, prob_order
    #
    df.sort_values(by=['datum','avd','startnr','häst'],inplace=True)
    vecko_df.sort_values(by=['datum','avd','startnr','häst'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    vecko_df.reset_index(drop=True,inplace=True)
    
    #### Check om utgått mellan varven
    vecko_sz = vecko_df[vecko_df.datum.str.slice(0,10)==dat].shape[0]
    df_sz =    df[df.datum==dat].shape[0]
    
    if vecko_sz>0 and vecko_sz != df_sz:
        print()
        print(f"OBS OBS! vecko_df har {vecko_sz} rader med df har {df_sz} rader. kolla att allt ok")
        vdf = vecko_df[vecko_df.datum.str.slice(0,10)==dat]
        print(vdf[~vdf.häst.isin(df.häst)][['datum','avd','startnr','häst']],'\n')

    # fas 1, 2, 3 innan första loppet. Fas 4 efter omgångens slut
    if fas==1:  # 15 minuter innan start
        # Ta bort ev rader med denna datum
        vecko_df=vecko_df[vecko_df.datum.str.slice(0,10)!=dat]

        # Lägg till efter sista raden
        vecko_df = pd.concat([vecko_df,df[['datum','avd','startnr','häst','vodds','prob_order']]])
        
    elif fas==2: # 10 min innan start
        # fyll i vodds2, proba_order2
        multiv = vecko_df.set_index(['datum', 'avd','startnr']).sort_index()
        multid = df.set_index(['datum', 'avd','startnr']).sort_index()
        
        multiv.vodds2.loc[dat] = multid.vodds
        multiv.prob_ord2.loc[dat] = multid.prob_order

        vecko_df = multiv.reset_index()
         
    elif fas==3: # 5 min innan start
        # fyll i vodds3, proba_order3
        multiv = vecko_df.set_index(['datum', 'avd','startnr']).sort_index()
        multid = df.set_index(['datum', 'avd','startnr']).sort_index()
        
        multiv.vodds3.loc[dat] = multid.vodds
        multiv.prob_ord3.loc[dat] = multid.prob_order

        vecko_df = multiv.reset_index()
 
    elif fas==4: # Efter omgången är klar och alla odds är de slutliga
        # fyll i vodds4, proba_order4
        multiv = vecko_df.set_index(['datum', 'avd','startnr']).sort_index()
        multid = df.set_index(['datum', 'avd','startnr']).sort_index()
        
        multiv.vodds4.loc[dat] = multid.vodds
        multiv.prob_ord4.loc[dat] = multid.prob_order

        vecko_df = multiv.reset_index()
 
    else:  # fel
        import sys
        sys.exit("FEL FEL FEL")
  

    cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
    vecko_df = vecko_df.reindex(columns=cols)

    vecko_df.sort_values(by=['datum','avd','startnr','häst'],inplace=True)

    return vecko_df

## Skapa V7-system

In [30]:
# SKAPA V75 RADER
def v75(df,modell, ant_rader=2):
    #,inplace=True)
    #df.reset_index(drop=True,inplace=True)

    #print(model.get_feature_importance(prettified=True))                    
    # selected_features = ['häst','kusk','bana','kön','spår','podds','dist','ålder','streck','vodds','bins','pris'] 
    selected_features = ['häst','kusk','bana','spår','bins','dist','ålder','streck','podds','vodds','pris'] 
    cat_features = ['häst','kusk','bana']

    proba = modell.predict_proba(df[selected_features])

    # Ordna proba per avdelning samt beräkna Kelly
    kassa=200
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    # Ta ut de 2 bästa per avd
    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1

    return(df)
    


In [31]:
def visa_v75(df,antal_rader):    
    df['p_o']=df.prob_order
    print(f"V75 {antal_rader}-raders")
    print(df[df.prob_order<=antal_rader][['avd','startnr','häst','vodds','p_o','proba','f','insats']])
    #print(df[df.prob_order<=antal_rader][['avd','startnr','häst','vodds','prob_order','proba','f','insats']])
    
def reserver(df, antal_rader):
    df['p_o']=df.prob_order
    print(f"Antal prob_order>{antal_rader} med insats>0")
    print(df[(df.prob_order>antal_rader) & (df.insats>0)] [['avd','startnr','häst','vodds','proba','p_o','f','insats']].sort_values(by='insats',ascending=False))
    df.drop('p_o',axis=1,inplace=True)

def visa_faser(vecko_df,dat,startnr=True):
    temp_df = vecko_df.copy()
    temp_df['X1'] = (temp_df.prob_order<=2)*1
    temp_df['X2'] = (temp_df.prob_ord2<=2)*1
    temp_df['X3'] = (temp_df.prob_ord3<=2)*1
    temp_df['X4'] = (temp_df.prob_ord4<=2)*1

    print('\nJämför faser')
    if startnr:
        print(temp_df[(temp_df.datum.str.slice(0,10) == dat) & ((temp_df.X1==1) | (temp_df.X2==1)| (temp_df.X3==1)| (temp_df.X4==1))][['avd','startnr','häst','vodds','vodds2','vodds3','vodds4','X1','X2','X3','X4']])
    else:
        print(temp_df[(temp_df.datum.str.slice(0,10) == dat) & ((temp_df.X1==1) | (temp_df.X2==1)| (temp_df.X3==1)| (temp_df.X4==1))][['avd','häst','vodds','vodds2','vodds3','vodds4','X1','X2','X3','X4']])            


# Main

In [32]:
##### Alternativ V75-rad #####
pd.set_option('display.width',100)
def alternativ_v75():
    global df, strukna

    def remove_features(df,remove_mer=[]):
        df.drop(['avd','startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True) #
        if remove_mer:
            df.drop(remove_mer,axis=1,inplace=True)
        
        return df
###############################################################################

    from catboost import CatBoostClassifier,Pool
    strukna = pd.DataFrame(columns=['datum','avd','startnr','häst','vodds'])

    modell = 'version 3/model_senaste'
    model = CatBoostClassifier()
    model.load_model(modell, format='cbm')

    ### scraing och predict ###
    df, strukna = vs.v75_scraping(history=True,resultat=False) 

    dfr = remove_features(df.copy()) ## för cat_features och pool
    cat_features=list(dfr.loc[:,dfr.dtypes=='O'].columns)
    pool = Pool(dfr,cat_features=cat_features)
    proba = model.predict_proba(pool)

    # Ordna proba per avdelning samt beräkna Kelly
    kassa=200
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    # Ta ut de 2 bästa per avd
    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1

    visa_v75(df, 2)
    reserver(df,2)

alternativ_v75()

omgång 1: https://www.atg.se/spel/2021-05-15/V75/
klickade på ANPASSA
anpassa klar - break
ant lopp 7
Ant priser 7
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 1 UMÅKER 2640 VOLTSTART ............
ant names,vodds,podds,rader,streck 15 15 15 15
AVD 2 UMÅKER 2140 VOLTSTART ...............
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 3 UMÅKER 2140 AUTOSTART ............
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 4 UMÅKER 2140 AUTOSTART ............
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 5 UMÅKER 1640 AUTOSTART ............
ant names,vodds,podds,rader,streck 10 10 10 10
AVD 6 UMÅKER 2140 AUTOSTART ..........
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 7 UMÅKER 1640 AUTOSTART ............

det tog 116.732 sekunder

startar Fixa mer
plac saknas är felaktig
V75 2-raders
    avd  startnr               häst  vodds  p_o     proba         f      insats
5     1        6  DIGITAL DOMINANCE  99.99    1  0.823510  0.821727  164.345418
8     1        9      DEAL 

In [33]:
def klassificering(df,proba=0.1,insats=0,antal_A=10):
    df_spel = df[(df.proba>proba)&(df.insats>insats)][['häst','avd','startnr','proba','prob_order','insats','streck','vodds']]

    antal = df_spel.shape[0]
    A_limit=df_spel.sort_values(by=['proba','avd','insats'],ascending=False).head(antal_A).proba.min()
    antal_B=antal-antal_A
    print('A_limit',round(A_limit,3))
    B_limit=df_spel.sort_values(by=['proba','avd','insats'],ascending=False).tail(antal_B).proba.min()
    print('B_limit',round(B_limit,3))

    print(f'Tot antal={len(df)}, antal_A={antal_A}, antal_B={antal_B}, antal_C={len(df)-(antal_A+antal_B)}')
    print()
    df_spel['ranking']='C'
    df_spel.loc[df_spel.proba>B_limit,'ranking'] = 'B'
    df_spel.loc[df_spel.proba>A_limit,'ranking'] = 'A'
    print(  'avd 1\n',df_spel[df_spel.avd==1][['häst','ranking','vann']])
    print('\navd 2\n',df_spel[df_spel.avd==2][['häst','ranking','vann']])
    print('\navd 3\n',df_spel[df_spel.avd==3][['häst','ranking','vann']])
    print('\navd 4\n',df_spel[df_spel.avd==4][['häst','ranking','vann']])
    print('\navd 5\n',df_spel[df_spel.avd==5][['häst','ranking','vann']])
    print('\navd 6\n',df_spel[df_spel.avd==6][['häst','ranking','vann']])
    print('\navd 7\n',df_spel[df_spel.avd==7][['häst','ranking','vann']])


df['vann']=False
# df.loc[(df.avd==1)&(df.startnr==3),'vann']=True
# df.loc[(df.avd==2)&(df.startnr==2),'vann']=True
# df.loc[(df.avd==3)&(df.startnr==4),'vann']=True
# df.loc[(df.avd==4)&(df.startnr==5),'vann']=True
# df.loc[(df.avd==5)&(df.startnr==10),'vann']=True
# df.loc[(df.avd==6)&(df.startnr==4),'vann']=True
# df.loc[(df.avd==7)&(df.startnr==4),'vann']=True
print(f"\nant A rätt:{len(df[df.vann & (df.ranking=='A')])}, ant B rätt:{len(df[df.vann & (df.ranking=='B')])}")

print('\n\nKandidater från Kelly (1 eller 2 får poäng=1 alt 4    eller tilldelas ranking=B) ')
print(df[df.ranking!='A'].sort_values(by='insats',ascending=False)[['avd','häst','ranking','proba','insats']])

A_limit 0.68
B_limit 0.106
Tot antal=85, antal_A=10, antal_B=57, antal_C=18

avd 1
                  häst ranking   vann
5   DIGITAL DOMINANCE       A  False
8       DEAL DONE ZET       A  False
3     AMORCER LEVALLO       B  False
6           ARCH LANE       B  False
1       ARIZONA DREAM       B  False
4       MISTER DONALD       B  False
11   JUST CELEBRATION       B  False
9          GAZZA B.R.       B  False

avd 2
                  häst ranking   vann
17         ARQUANA AS       A  False
14             NAVONA       B  False
22         MOON ZAPPA       B  False
20        GALANT SISU       B  False
25           QEEN ROC       B  False
13        KAILUA BOKO       B  False
23  ANNA MAUDS LASSIE       B  False
12     VÄSTERBO VIXEN       B  False
24           LA NORNA       B  False
21       MELLBY GUAVA       B  False
19                AVA       B  False
16      ALFAS LEONORE       C  False

avd 3
                   häst ranking   vann
31       SHORT IN CASH       B  False
30        

In [113]:
ettor=2 # utöver de 7
tvåor=4
treor=8
fyror=7
def sätt_poäng(df,extra_ettor=2,tvåor=4,treor=8,fyror=7):
    df['poäng'] = 15
    df.loc[df.prob_order==1,'poäng'] = 1
    df.sort_values(by='proba',ascending=False,inplace=True)
    ix=df[df.poäng!=1].iloc[:ettor,:].index
    df.loc[ix,'poäng']=1
    ix=df[df.poäng!=1].iloc[:tvåor,:].index
    df.loc[ix,'poäng']=2
    ix=df[df.poäng>2].iloc[:treor,:].index
    df.loc[ix,'poäng']=3
    ix=df[df.poäng>3].iloc[:fyror,:].index
    df.loc[ix,'poäng']=4

    # min Kelly-häst
    df.sort_values(by='insats',ascending=False,inplace=True)
    ix=df[df.poäng>2].iloc[:1,:].index
    df.loc[ix,'poäng']=2
    print(f"\nmin Kelly-häst: {df.loc[ix,'häst'].values} avd={df.loc[ix,'avd'].values[0]} insats={round(df.loc[ix,'insats'].values[0],2)}\n")

    print(df.poäng.value_counts())

    return df


min Kelly-häst: ['G.K.JUSTUS'] avd=4 insats=124.13

15    57
1      9
4      7
3      7
2      5
Name: poäng, dtype: int64

avd=1
   startnr               häst  poäng
5        6  DIGITAL DOMINANCE      1
8        9      DEAL DONE ZET      1
2        3      MADE OF STARS      3
3        4    AMORCER LEVALLO      4

avd=2
    startnr         häst  poäng
17        6   ARQUANA AS      1
14        3       NAVONA      2
22       11   MOON ZAPPA      3
20        9  GALANT SISU      3

avd=3
    startnr           häst  poäng
27        1    MISSLE HILL      1
31        5  SHORT IN CASH      4

avd=4
    startnr              häst  poäng
49       11   AMALENCIUS B.B.      1
47        9  GLOBAL ATTENTION      2
46        8        G.K.JUSTUS      2
41        3            CUENCA      4
45        7       BERRA STEEL      4

avd=5
    startnr             häst  poäng
59        9    VIKING BRODDE      1
55        5       ALCIDE ROC      2
51        1           KAPTAH      3
62       12  SUMMIT IN SIGHT

In [116]:
!pip install -U streamlit

  Using cached base58-2.1.0-py3-none-any.whl (5.6 kB)
  Using cached blinker-1.4.tar.gz (111 kB)
  Using cached validators-0.18.2-py3-none-any.whl (19 kB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
  Using cached pydeck-0.6.2-py2.py3-none-any.whl (4.2 MB)
  Using cached altair-4.1.0-py3-none-any.whl (727 kB)
  Using cached GitPython-3.1.14-py3-none-any.whl (159 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached tzlocal-2.1-py2.py3-none-any.whl (16 kB)
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13454 sha256=3384ebae2c6cffc9752519127dce08be41ed670fe9576cc7201489f03dca1647
  Stored in directory: c:\users\peter\appdata\local\pip\cache\wheels\b7\a5\68\fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4
Successfully built blinker
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --n

In [118]:

import streamlit as st
st.subheader('Avd 1')
st.write(df[(df.avd==1)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])
# print('\navd=1')
# print(df[(df.avd==1)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=2')
# print(df[(df.avd==2)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=3')
# print(df[(df.avd==3)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=4')
# print(df[(df.avd==4)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=5')
# print(df[(df.avd==5)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=6')
# print(df[(df.avd==6)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])

# print('\navd=7')
# print(df[(df.avd==7)&(df.poäng<15)].sort_values(by=['poäng'])[['startnr','häst','poäng']])


2021-05-11 15:37:34.860 WARNING root: 
  command:

    streamlit run C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [114]:
def sätt_poäng2(df,ettor=9, tvåor=4, treor=8, fyror=7 ):
    df['poäng'] = 15
    print('Sorterat på prob_order och avd (alla prob_order==1 per avd får poäng=1')
    df[df.prob_order==1,'poäng'] = 1
    # print(df.sort_values(by=['prob_order','avd'])[['häst','avd','prob_order','insats']].head(7))

    print('\nSorterat på proba där prob_order>1')
    print('De två första får poäng=1 - hur många skall få poäng=2-6?')
    df[df.poäng>1].sort_values(by='proba',ascending=False).iloc[:2,'poäng']=1
    # print(df[df.prob_order>1].sort_values(by='proba',ascending=False)[['häst','avd','prob_order','insats']].head(2 ))

    print('De fyra nästa (plus en Kelly) får poäng=2')
    print(df[df.prob_order>1].sort_values(by='proba',ascending=False)[['häst','avd','prob_order','insats']].iloc[2:5,:])

    print('De åtta nästa får poäng=3')
    print(df[df.prob_order>1].sort_values(by='proba',ascending=False)[['häst','avd','prob_order','insats']].iloc[5:14,:])

    print('De sju nästa (samt en Kelly?) får poäng=4')
    print(df[df.prob_order>1].sort_values(by='proba',ascending=False)[['häst','avd','prob_order','insats']].iloc[15:23,:])


In [11]:
#### Orginal-version ####
déf orginal_version(): 
    vecko_df=pd.read_csv('vecko_df.csv')
    pd.set_option('mode.chained_assignment',None)

    modell = 'modeller_streck_100/model_senaste'
    from catboost import CatBoostClassifier
    model = CatBoostClassifier()
    model.load_model(modell, format='cbm')

    init=False
    spara=True
    fas=4
    after = fas==4

    omg_df = pd.read_csv('omg_att_spela_link.csv' )

    if fas==1:
        vecko_df=pd.read_csv('vecko_df.csv')

    # scraping
    if init:
        # starta webdriver
        driver_s = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')
        driver_s.get(omg_df.Link[0])
        driver_s.implicitly_wait(5) # seconds
        driver_s.fullscreen_window()
        
        import os
        input('anpassa (krPerstart, dist/spår, vodds, podds) och sedan tryck enter här')

    #### TEMPORÄRT - använd df frå alternativ_v75() ###    
    #df,strukna = scraping(omg_df, driver_s)
    ### ANVÄND df FRÅN alternyiv_v75 ####

    if vecko_df.häst.isin(strukna.häst).sum() !=0:
        print('ta bort strukna från vecko_df:')
        vecko_df[~vecko_df.häst.isin(strukna.häst)][['datum','avd','startnr','häst','vodds']]
        vecko_df = vecko_df[~vecko_df.häst.isin(strukna)]


    # V75
    pd.set_option('display.width',100)
    antal_rader=2
    df=v75(df, model, antal_rader)
    print()
    visa_v75(df,antal_rader)      
    print()
    reserver(df,antal_rader)
    # addera vodds och jämför
    dat = omg_df.Link[0].split('spel/')[1][0:10]  #gräv ut datum
    vecko_df = addera_vodds(df,vecko_df, dat, fas)

    print('\n\n')
    visa_faser(vecko_df,dat)


    if spara:
        # Avsluta och Spara till vecko_df.csv
        vecko_df.to_csv('vecko_df.csv',index=False)
        



V75 2-raders
    avd  startnr               häst  vodds  p_o     proba         f     insats
5     1        6  DIGITAL DOMINANCE  99.99    1  0.327593  0.320800  64.160088
8     1        9      DEAL DONE ZET  99.99    2  0.247869  0.240270  48.054096
17    2        6         ARQUANA AS  99.99    1  0.248006  0.240410  48.081926
14    2        3             NAVONA  99.99    2  0.150233  0.141648  28.329644
27    3        1        MISSLE HILL   1.00    1  0.754430      -inf        NaN
31    3        5      SHORT IN CASH  99.99    2  0.091939  0.082766  16.553187
49    4       11    AMALENCIUS B.B.  34.85    1  0.146228  0.121006  24.201104
47    4        9   GLOBAL ATTENTION  32.16    2  0.135033  0.107274  21.454806
59    5        9      VIKING BRODDE  99.99    1  0.209730  0.201747  40.349368
51    5        1             KAPTAH  99.99    2  0.155698  0.147169  29.433737
68    6        6    DON FANUCCI ZET  99.99    1  0.436737  0.431047  86.209467
71    6        9          HAIL MARY  9

## När allt är klart

In [12]:
 driver_s.quit()

NameError: name 'driver_s' is not defined

# Extra grejer och tester

In [ ]:
game_tab=driver_s.find_elements_by_class_name('game-table')[1:]                       ## alla lopp utan rubriker
rader = game_tab[1].find_elements_by_class_name("startlist__row")                     ## alla rader i loppet
dat = omg_df.Link[0].split('spel/')[1][0:10]
rad=rader[3]
for rad in rader:
    print(rad.find_elements_by_class_name("earningsPerStart-col")[0].text )  ##  kr/start i lopp 1 utan rubrik


In [ ]:
df=pd.DataFrame(vdict)  
df.isna().sum().sum()

In [ ]:
##    Visa V75 mm    ##
antal_rader=2
df=v75(df, model, antal_rader)

visa_v75(df, 2)
print()
reserver(df,2)
print()
visa_faser(vecko_df,dat,startnr=False)

In [ ]:
# Vinnare
print('Vinnare')
dat = omg_df.Link[0].split('spel/')[1][0:10]  #gräv ut datum
df[(df.f>0) & (df.datum==dat)][['avd','startnr','häst','vodds','prob_order','proba','f','insats']]

In [ ]:
utdelning = pd.read_csv('utdelning.csv')
utdelning.drop_duplicates(['datum'],inplace=True)

utdelning.sort_values(by=['datum'],inplace=True)
utdelning.reset_index(drop=True,inplace=True)
utdelning.to_csv('utdelning.csv',index=False)
   

In [ ]:
from catboost import CatBoostClassifier
modell='modeller_all_100/model_2021-02-20'
print(modell)
model = CatBoostClassifier()
model.load_model(modell, format='cbm')
model.get_feature_importance(prettified=True)

In [ ]:
## Ta bort en hel omgång ##

vecko_df=vecko_df[vecko_df.datum!=dat]
df=df[df.datum!=dat]
print(df.shape)
vecko_df.shape


In [ ]:
# Spara
vecko_df.to_csv('vecko_df.csv',index=False)

In [ ]:
# Systemets kostnad #

tvåor=6
treor=1
fyror=0
femmor=0

if tvåor+treor+fyror+femmor != 7:
    print ("FEL FEL", tvåor+treor+fyror+femmor, "borde vara 7")
else:
    print(f"systemetss kostnad: {5**femmor * 4**fyror * 3**treor  * 2**tvåor/2} kr")

# Testa ny funktion för get (före omg resp efter)

In [ ]:
driver_s = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')
omg_df = pd.read_csv('omg_att_spela_link.csv' )
driver_s.get(omg_df.Link[0])
driver_s.implicitly_wait(5) # seconds

import os
input('sätt fullscreen och anpassa sedan tryck enter här')



In [ ]:
import scraping as scr

avd=7
i=12

namn = scr.getNamn(driver_s, str(avd), i, after=True)
kön = scr.getKön(driver_s, str(avd), i, after=True)
ålder = scr.getÅlder(driver_s, str(avd), i, after=True)
bana = scr.getBana(driver_s, str(avd), after=True)
spår = scr.getSpår(driver_s, str(avd), i, after=True)  # spår och distans
kusk = scr.getKusk(driver_s, str(avd), i, after=True)
vodds = scr.getVodds(driver_s, str(avd), i, after=True)
podds = scr.getPodds(driver_s, str(avd), i, after=True)
kr = scr.getKrPerStart(driver_s, str(avd), i, after=True)
print(namn, kusk, kön, ålder, bana,'spår',spår,'vodds',vodds,'podds', podds, 'kr',kr)

In [ ]:
df[df.datum==dat].sort_values(by=['avd','proba','startnr'],ascending=False).tail(60)

In [ ]:
#vecko_df=pd.read_csv('vecko_df.csv')
vecko_df[vecko_df.datum==dat]

In [ ]:
# vecko_df = pd.concat([vecko_df,df[['datum','avd','startnr','häst','vodds','prob_order']]])
# cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
# vecko_df = vecko_df.reindex(columns=cols)
 
vecko_df.sort_values(by=['datum','avd','startnr','häst'],inplace=True)
        

temp=df[['datum','avd','startnr','häst','vodds','prob_order']]
temp.columns=['datum','avd','startnr','häst','vodds2','prob_ord2']
temp 
cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']      
vecko_temp =pd.merge(vecko_df[vecko_df.datum==dat][cols], temp[['datum','avd','startnr','häst','vodds2','prob_ord2']], on = ["datum","avd","nr","häst"], how = "outer") 

cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
vecko_temp = vecko_temp.reindex(columns=cols)

vecko_df = pd.concat([vecko_df[vecko_df.datum!=dat],vecko_temp[['datum','avd','startnr','häst','vodds2','prob_order2']]])



In [ ]:
vecko_df

In [ ]:
df

In [ ]:
temp=df[['datum','avd','startnr','häst','vodds','prob_order']]
temp.columns=['datum','avd','startnr','häst','vodds2','prob_ord2']
    
cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']      
vecko_temp = pd.merge(vecko_df[vecko_df.datum==dat][cols], temp[['datum','avd','startnr','häst','vodds2','prob_ord2']], on = ["datum","avd","nr","häst"], how = "outer") 

vecko_df = pd.concat([vecko_df[vecko_df.datum!=dat],vecko_temp])
 
vecko_df.sort_values(by=['datum','avd','startnr','häst'],inplace=True)
  
cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
vecko_df=vecko_df.reindex(columns=cols)
        

In [ ]:
vecko_org = vecko_df.copy()

In [ ]:
multiv = vecko_df.set_index(['datum', 'avd','startnr']).sort_index()
multid = df.set_index(['datum', 'avd','startnr']).sort_index()

multiv.vodds2.loc[dat] = multid.vodds
multiv.prob_ord2.loc[dat] = multid.prob_order
multiv.reset_index(inplace=True) 
cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
multiv.reindex(columns=cols)



#cols = ['datum','avd','startnr', 'häst', 'vodds', 'prob_order', 'vodds2', 'prob_ord2', 'vodds3', 'prob_ord3', 'vodds4', 'prob_ord4']
#v.reindex(columns=cols)

print(multiv)

In [ ]:
multiv = vecko_df.set_index(['datum', 'avd','startnr']).sort_index()
multid = df.set_index(['datum', 'avd','startnr']).sort_index()

multiv.vodds2.loc[dat] = multid.vodds
multiv.prob_ord2.loc[dat] = multid.prob_order

vecko_df = multiv.reset_index()         

In [ ]:
vecko_df.drop_duplicates(subset=['datum','häst'],keep='last',inplace=True)


In [ ]:
vecko_df[vecko_df.datum==dat]

In [ ]:
kompl = pd.read_csv('mer_komplett.csv')
groups= kompl.groupby(['datum'])['häst'].count()

groups[groups<50]


In [ ]:
komplett